In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pyperclip
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pprint import pprint
from tqdm import tqdm, notebook
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
path = r'C:\Users\재욱\Downloads\chromedriver.exe'
driver = webdriver.Chrome(path)
driver.implicitly_wait(3)
country_url = "https://www.brandsoftheworld.com/logos/countries" 
driver.get(country_url)

xpath = '//*[@id="primaryInner"]/div/ul'
division = driver.find_element_by_xpath(xpath)
country_list = division.find_elements_by_tag_name('a') # List형

country_url_list = []
for country_url in country_list:
    country_url_list.append(country_url.get_attribute('href'))

## 빈 딕셔너리 (dict_status) 선언 후 JSON 파일 먼저 만들기

In [16]:
dict_status = {}
with open("brands_of_the_world.json", "w") as upload_file:
    json.dump(dict_status, upload_file)

## JSON with image download

In [20]:
# 빈 딕셔너리 (dict_status) 선언 후 JSON 파일 먼저 만들기
try:
    XPATH_COUNTRY_TABLE = '//*[@id="primaryInner"]/div/div[1]'
    XPATH_COUNTRY_NAME = '//*[@id="primary"]/h1/span'
    print("지난 status 상황 : ")
    with open("brands_of_the_world.json", "r") as download_file:
        dict_status = json.loads(download_file.read())
    pprint(dict_status)
    print('+-----------------------------------------------------------------+')

    # COUNTRY_NUMBER 대신 enumerate를 쓰는 연습
    for idx , country_path in enumerate(notebook.tqdm(country_url_list, desc='iterate range 100')):
        driver.get(country_path)
        country_name = driver.find_element_by_xpath(XPATH_COUNTRY_NAME).text
        
        # 해당 국가의 key-value가 없을 경우. (해당 국가 페이지 처음 방문)
        if country_name not in dict_status.keys():
            dict_status[country_name] = {"status" : 'NOT DONE', "latest_page" : -1} # 현재 국가 save status 초기화
            page_num = 0
        # 해당 국가의 key-value가 있을 경우. DONE이든 NOT DONE이든 (해당 국가 페이지 방문한 적 있음)
        else: 
            page_num = dict_status[country_name]["latest_page"]
            
        # 해당 국가의 value의 status key가 'NOT DONE'일 경우(해당 국가 페이지 쭉 방문해야함.)    
        if dict_status[country_name]["status"] != 'DONE':   
            
            while True:
                driver.get(f'{country_path}?page={page_num}') #f string 사용!
                division = driver.find_element_by_xpath(XPATH_COUNTRY_TABLE)
                name_list = division.find_elements_by_tag_name('li') # List형
                image_url_list = division.find_elements_by_tag_name('img') # 이미지 URL
                
                if len(image_url_list) == 0:
                    break
                
                img_folder_path = './brands_of_the_world/' + country_name + '_company_logo/'    
                os.makedirs(img_folder_path, exist_ok=True)
                
                for i, image_url in enumerate(image_url_list):
                    try:
                        with urllib.request.urlopen(image_url.get_attribute('src')) as f:
                            with open(img_folder_path + name_list[i].text +'.jpg','wb') as h:
                                image = f.read()
                                h.write(image)
                    except Exception as e:
                        print(e)
                        
                print(f"{country_name}의 {page_num}번째 page 이미지 저장 완료")
                dict_status[country_name]["latest_page"] +=1
                page_num +=1
                    
            dict_status[country_name]["status"] = 'DONE'
            with open("brands_of_the_world.json", "w") as upload_file:
                    json.dump(dict_status, upload_file)
            pprint(dict_status)
            print(f'{idx}번째 나라 {country_name}의 모든 이미지 저장 완료')
            print('+-----------------------------------------------------------------+')
         
except Exception as e:
    print(e)
except KeyboardInterrupt:
    print("[KeyboardInterrupt] 현재까지의 status :")
    with open("brands_of_the_world.json", "w") as upload_file:
                    json.dump(dict_status, upload_file)
    pprint(dict_status)
    print("\n")
except:
    print('Other error')

지난 status 상황 : 
{'Afghanistan': {'latest_page': 2, 'status': 'DONE'},
 'Albania': {'latest_page': 3, 'status': 'NOT DONE'}}
+-----------------------------------------------------------------+


iterate range 100:   0%|          | 0/240 [00:00<?, ?it/s]

Albania의 3번째 page 이미지 저장 완료
[Errno 2] No such file or directory: './brands_of_the_world/Albania_company_logo/24/7 Rental Cars.jpg'
Albania의 4번째 page 이미지 저장 완료
Albania의 5번째 page 이미지 저장 완료
Albania의 6번째 page 이미지 저장 완료
[KeyboardInterrupt] 현재까지의 status :
{'Afghanistan': {'latest_page': 2, 'status': 'DONE'},
 'Albania': {'latest_page': 7, 'status': 'NOT DONE'}}


